# Part I — Foundations  
## 1. Web Development Essentials (Crash Course) — Rewritten With Full Explanations

This chapter is the “why everything in Django looks the way it does” chapter. If you
understand this deeply, Django’s URLs, views, templates, forms, sessions, CSRF, and
REST APIs will feel logical instead of magical.

---

## 1.0 Learning Outcomes (What you must be able to do after this chapter)

You should be able to:

1. Explain, in detail, what happens from the moment you type a URL into a browser
   to the moment you see a page.
2. Read an HTTP request/response and correctly interpret:
   - method, path, query string, headers, body
   - status code meaning and consequences
3. Explain what a URL is, what a query string is, what `?` means, what `&` means,
   and what gets sent to the server vs what does not.
4. Describe why pagination exists, why it’s commonly implemented via query
   parameters like `?page=2`, and what its limitations are (offset pagination
   issues, cursor pagination, etc.).
5. Explain cookies vs sessions and how “being logged in” works.
6. Recognize core web security threats (CSRF, XSS, SQL injection) and explain how
   Django’s defaults address them.

---

## 1.1 The Request–Response Lifecycle (The Core Web Loop)

At the most fundamental level, **web applications are request/response machines**.

### 1.1.1 What happens when you visit a website (step-by-step)

When you navigate to:

```text
https://example.com/articles/?page=2
```

a typical sequence is:

1. **DNS lookup**: Browser finds the IP address for `example.com`.
2. **TCP connection** (or QUIC for HTTP/3): A network connection is established.
3. **TLS handshake** (because it’s HTTPS): encryption keys are negotiated and the
   server proves its identity with a certificate.
4. **HTTP request is sent**: browser sends a request message containing method,
   path, headers, and sometimes a body.
5. **Reverse proxy/web server** (often Nginx/Caddy/load balancer) receives it:
   - may serve static files directly (images, CSS)
   - may forward dynamic requests to Django (Gunicorn/Uvicorn)
6. **Django processes the request**:
   - middleware runs (security, sessions, etc.)
   - URL routing finds the right view
   - view executes business logic (DB queries, validations)
   - response object is created (HTML/JSON/redirect)
7. **HTTP response is sent back**: includes status code, headers, and body.
8. **Browser renders**:
   - if HTML: parses it, fetches CSS/JS/images (each is more HTTP requests)
   - if JSON: a JS app might render UI from it

### 1.1.2 Why this matters for Django
Django is built around this loop. Almost every Django concept maps to a part of it:

- **URLs**: deciding *which code* handles a request
- **Views**: *the code* that builds a response
- **Templates**: generating HTML
- **Models/ORM**: reading/writing the database
- **Middleware**: cross-cutting behavior around every request (sessions, CSRF, etc.)

---

## 1.2 HTTP: Anatomy of Requests and Responses (With Full Meaning)

HTTP is a protocol for sending messages. You’ll frequently debug apps by reading raw
HTTP, so you must understand each part.

### 1.2.1 The structure of an HTTP request

An HTTP request contains:

1. **Request line**: method + path + protocol version
2. **Headers**: key/value metadata
3. **Blank line**
4. **Body** (optional): data being sent to server

Example (raw-ish):

```text
POST /login/?next=/dashboard/ HTTP/1.1
Host: example.com
User-Agent: Mozilla/5.0 ...
Accept: text/html,application/xhtml+xml
Content-Type: application/x-www-form-urlencoded
Content-Length: 33
Cookie: csrftoken=abc; sessionid=xyz

username=alice&password=secret
```

Let’s interpret it:

- `POST` means “I’m submitting data that changes server state” (often).
- `/login/` is the **path** (which route should handle it).
- `?next=/dashboard/` is the **query string**, usually optional behavior.
- `Host` says which domain (important when many sites share one server).
- `Accept` says what response formats the client can handle.
- `Content-Type` says what format the request body is.
- `Cookie` contains previously stored cookies (session state, CSRF token, etc).
- Body contains form data.

### 1.2.2 The structure of an HTTP response

A response contains:

1. **Status line**: protocol + status code + reason phrase
2. **Headers**
3. **Blank line**
4. **Body**

Example:

```text
HTTP/1.1 302 Found
Location: /dashboard/
Set-Cookie: sessionid=newsession; Path=/; HttpOnly; Secure; SameSite=Lax
Content-Length: 0
```

Interpretation:

- `302 Found` = redirect (browser should go to another URL)
- `Location` header tells where
- `Set-Cookie` updates stored browser cookie
- empty body

---

## 1.3 HTTP Methods Deeply (Not Just “GET vs POST”)

### 1.3.1 Safe vs idempotent (industry terms you must know)

- **Safe method**: should not change server state.
  - GET, HEAD are intended to be safe.
- **Idempotent method**: repeating it produces the same result as doing it once.
  - GET is idempotent.
  - PUT is idempotent (replace resource with same representation repeatedly).
  - DELETE is idempotent in the sense “after first delete, it remains deleted”.
  - POST is generally **not** idempotent (two POSTs often create two records).

Why you care:
- browsers, proxies, CDNs, and prefetchers may retry or repeat requests.
- caching mechanisms assume GET is safe.

### 1.3.2 Practical meaning in Django apps

- Use **GET** for:
  - listing pages
  - detail pages
  - search pages
  - pagination links
- Use **POST** for:
  - login
  - signup
  - create/update form submissions
  - “actions” like “like”, “follow”, “pay”, etc.

A common industry guideline:
- If clicking a link triggers a destructive action (delete), it’s a red flag because
  links are GET requests. Destructive actions should require POST + CSRF protection.

---

## 1.4 Status Codes (What They Really Signal)

Status codes are not just “numbers”; they inform browser behavior, caching, retries,
and API clients.

### 1.4.1 The categories
- 2xx: success
- 3xx: redirect
- 4xx: client made a bad request or lacks permissions
- 5xx: server error

### 1.4.2 Important codes with realistic Django implications

- **200 OK**: normal response
- **201 Created**: common for API creation
- **204 No Content**: success with no body (common after delete)
- **301/302**: redirect
  - Django’s `redirect(...)` is typically 302.
- **400 Bad Request**: invalid input (API), malformed request
- **401 Unauthorized**: “you are not authenticated”
- **403 Forbidden**:
  - authenticated but not allowed
  - CSRF failure is often surfaced as 403
- **404 Not Found**: wrong URL or object missing (Django uses 404 often)
- **405 Method Not Allowed**: route exists but method not allowed
- **500**: unhandled error in your code

---

## 1.5 URLs, Paths, and Query Strings (Including What `?` Means)

This is where your pagination question fits.

### 1.5.1 URL anatomy (what gets sent to server)

Example:

```text
https://example.com:443/products/42/?page=2&sort=price#reviews
```

Breakdown:

- `https` (scheme): determines protocol + security expectations.
- `example.com` (host): domain.
- `:443` (port): usually omitted; default HTTPS port is 443.
- `/products/42/` (path): identifies the resource or page.
- `?page=2&sort=price` (query string): optional parameters that modify behavior.
- `#reviews` (fragment): **not sent to the server**. Used only by browser to jump
  within a page.

### 1.5.2 What does `?` mean?

`?` marks the beginning of the **query string** section of a URL.

- Everything **before** `?` is the path.
- Everything **after** `?` (until `#` if present) is query parameters.

Example:

```text
/path/to/page?key=value&key2=value2
```

- Query parameters are typically `name=value` pairs.
- `&` separates multiple parameters.

So:

```text
?page=2&sort=price
```

means:
- parameter `page` has value `2`
- parameter `sort` has value `price`

### 1.5.3 Why query parameters exist (what they’re for)

Query parameters are used for **optional, non-identifying inputs** that modify how a
resource is presented.

Classic uses:
- pagination (`?page=2`)
- filtering (`?status=published`)
- sorting (`?sort=price`)
- searching (`?q=django`)
- toggling expansions (`?include=comments`)
- tracking/analytics (`?utm_source=...`)

Why not always put these in the path?
- The path typically identifies “what resource is this?”
- The query string typically describes “how do you want it returned?”

Example difference:

- `/products/` → “the products collection”
- `/products/?page=2` → “products collection, second page”

This separation improves:
- predictability
- caching behavior
- link sharing
- standard tooling compatibility

### 1.5.4 URL encoding (important when values contain spaces/symbols)

Query strings use URL encoding. For example:

- space may become `%20` (or `+` in form encoding contexts)
- `&` and `=` have special meaning, so they must be encoded if part of a value

Example:

```text
?q=django orm
```

is not safe as-is. It becomes:

```text
?q=django%20orm
```

You’ll see this in browsers automatically.

---

## 1.6 Pagination: Why It Exists, Why `?page=2` is Common, and Limitations

### 1.6.1 What pagination is
Pagination means returning a large list in smaller chunks (“pages”), such as:

- Page 1: items 1–20
- Page 2: items 21–40
- Page 3: items 41–60

### 1.6.2 Why pagination exists (real reasons in production)

Without pagination:
- The server might try to fetch and serialize/render thousands of items.
- The database query may be slow.
- The response could be huge (MBs), slow to download, slow to render.
- The browser becomes sluggish.
- Mobile devices/data plans suffer.

Pagination is both a **performance feature** and a **user experience feature**:
- users don’t want an endless wall of content
- they want navigation and “where am I?” context

### 1.6.3 Why `?page=2` is structured like that (and why it’s the default pattern)

`?page=2` is a query parameter because:

1. Pagination modifies how you view the same resource collection (`/articles/`).
2. The underlying “resource” is still the articles collection; you’re not accessing a
   different kind of resource, just a different slice.
3. It’s easy to create links:
   - `/articles/?page=1`
   - `/articles/?page=2`
4. It plays well with caching/CDNs and with bookmarking/shareable URLs.
5. It composes cleanly with other query parameters:

```text
/articles/?tag=django&sort=newest&page=3
```

This composability is a major industry reason.

### 1.6.4 How pagination works conceptually (offset pagination)

Most classic pagination maps page number to an offset:

- page size = 20
- page 1 offset = 0
- page 2 offset = 20
- page 3 offset = 40

General formula:

- offset = (page - 1) * page_size

So `?page=3` with size 20 means “skip 40, return next 20”.

This is easy and user-friendly, but has limitations.

### 1.6.5 Limitations of page-number pagination (important!)

#### Limitation A: Data changes cause duplicates/missing items
If new records are inserted while the user is paging:

- User loads page 1 (items A–T)
- New items are inserted at the top
- User loads page 2; offset-based logic now shifts results
- User may see duplicates or miss items

This is common in feeds (news, social timelines).

#### Limitation B: Large offsets can be slow in databases
For many databases, `OFFSET 100000` can get expensive because the DB may have to
scan/skip many rows.

This can become a performance bottleneck at scale.

#### Limitation C: Page boundaries are arbitrary
If sorting changes, page membership changes. This isn’t “wrong,” but it affects UX
consistency.

### 1.6.6 Alternatives: limit/offset vs cursor pagination

#### (1) Limit/offset pagination
Common in APIs:

```text
GET /api/articles?limit=20&offset=40
```

Pros:
- explicit and flexible
- easy for infinite scroll (“load next 20”)

Cons:
- same shifting/duplication problem if data changes
- large offset performance issues still exist

#### (2) Cursor pagination (a more scalable approach)
Often looks like:

```text
GET /api/articles?cursor=eyJpZCI6MTIzfQ==&page_size=20
```

or:

```text
GET /api/articles?after=2026-02-01T10:00:00Z&limit=20
```

Instead of “page 2”, you say “give me items after this cursor”.

Pros:
- stable pagination even as new items appear (if designed correctly)
- better performance at scale (no huge OFFSET)

Cons:
- harder to implement and debug
- can be less user-friendly for “jump to page 50”
- cursors can be opaque and require consistent ordering

### 1.6.7 What is `page_size` and why might you expose it?
`page_size` is “how many items per page”.

Example:

```text
/articles/?page=2&page_size=50
```

Why it’s useful:
- some users want “show 100 per page”
- internal admin tools often want bigger pages
- API clients may want fewer items per request for speed

Why it’s dangerous to expose without limits:
- attackers can request huge page sizes (DoS vector)
- your server may generate enormous responses

Industry practice:
- allow `page_size` but cap it (e.g., max 100)

---

## 1.7 Headers: Why They Exist and Which Ones You’ll Actually Use

### 1.7.1 `Content-Type` vs `Accept` (common confusion)

- `Content-Type`: describes the format of the **body you are sending**.
  - Example: client sends JSON:
    - `Content-Type: application/json`
- `Accept`: describes what formats you want **back**.
  - Example: client wants JSON:
    - `Accept: application/json`

This is especially important for APIs and for debugging “why is this returning HTML?”

### 1.7.2 Common `Content-Type` values
- `text/html; charset=utf-8` → webpages
- `application/json` → JSON APIs
- `application/x-www-form-urlencoded` → classic form submissions
- `multipart/form-data` → file uploads + form fields

When you later use Django forms:
- Browser will submit form bodies as `application/x-www-form-urlencoded`
- If there are file inputs, it becomes `multipart/form-data`

---

## 1.8 HTML Pages vs JSON APIs (With Practical Examples)

### 1.8.1 Server-rendered HTML: how it feels to users
Request:

```text
GET /articles/hello-world/
Accept: text/html
```

Response body: HTML.

Browser:
- renders HTML
- then loads CSS/JS/images (more HTTP requests)

This is Django’s “classic” style: templates + forms.

### 1.8.2 JSON API: how it feels to clients
Request:

```text
GET /api/articles/42/
Accept: application/json
```

Response body:

```json
{
  "id": 42,
  "title": "Hello World",
  "author": "alice",
  "created_at": "2026-02-04T10:00:00Z"
}
```

Client:
- could be a mobile app
- could be a React frontend
- could be another backend service

### 1.8.3 You can do both (common in real companies)
A single Django project may:
- serve HTML for the public site
- serve JSON for mobile apps or internal tools
- provide webhooks endpoints for third-party integrations

---

## 1.9 Cookies and Sessions (What “Logged In” Actually Means)

### 1.9.1 Cookies: what they are, precisely
A cookie is:
- stored by the browser,
- associated with a domain,
- sent automatically with requests (under rules).

Server sets:

```text
Set-Cookie: sessionid=abc123; Path=/; HttpOnly; Secure; SameSite=Lax
```

Browser later sends:

```text
Cookie: sessionid=abc123
```

### 1.9.2 Why cookies are used for sessions
HTTP is stateless by default. Cookies allow the server to “recognize” a browser
between requests.

Most Django sites use a cookie like `sessionid` to store a session identifier.

### 1.9.3 Sessions: where the state lives
In a session-based system:

- Browser stores: session ID (small)
- Server stores: session data (potentially larger)

Conceptually:

```text
sessionid = "abc123"
server_session_store["abc123"] = {"user_id": 9, "cart": [42, 99]}
```

Django supports multiple session stores:
- database-backed
- cached (Redis/memcache)
- file-based
- signed cookies (data stored client-side but cryptographically signed)

You’ll learn the tradeoffs later; for now, remember:
- **cookie identifies the session**
- **session contains the state**

### 1.9.4 Cookie security attributes (why they matter)
- `HttpOnly`: prevents JS from reading the cookie (helps against XSS stealing).
- `Secure`: cookie only sent over HTTPS (prevents leakage over HTTP).
- `SameSite=Lax/Strict/None`: controls cross-site sending behavior (CSRF impact).

Industry standard:
- session cookies should be `HttpOnly` and `Secure` in production.

---

## 1.10 REST Basics (Why APIs Often Look Like They Do)

REST is a style of mapping resources to URLs and using HTTP methods consistently.

Typical pattern:

- `GET /api/articles/` → list
- `POST /api/articles/` → create
- `GET /api/articles/42/` → retrieve
- `PATCH /api/articles/42/` → update partial
- `DELETE /api/articles/42/` → delete

Why this is common:
- consistent and predictable
- tooling works well (docs, clients)
- maps cleanly to CRUD in databases

Not every API is purely REST, but most professional Django APIs follow these
conventions (especially with Django REST Framework).

---

## 1.11 Security Essentials (With Realistic “Why”)

### 1.11.1 CSRF: why it exists (in plain language)
Because browsers automatically attach cookies to requests, an attacker can trick a
logged-in user’s browser into sending a request to your site.

CSRF protection proves:
- the request originated from pages your server rendered (or your JS app), not from
  an attacker’s site.

Django’s CSRF protection:
- expects a CSRF token in POST/unsafe requests
- rejects requests without a valid token

### 1.11.2 XSS: what it is and why escaping matters
XSS happens when untrusted input is inserted into HTML/JS in a way that executes as
code.

Django templates auto-escape by default, which prevents many XSS cases. But you can
still create XSS by:
- marking unsafe strings as safe
- inserting untrusted data into JS contexts incorrectly
- rendering rich HTML from users without sanitization

### 1.11.3 SQL injection: what it is and why ORM helps
SQL injection occurs when user input is concatenated into SQL.

Django ORM prevents most injection because it parameterizes queries (sends values
separately from SQL structure). But risk returns if you build raw SQL incorrectly.

---

## 1.12 Hands-On Mini Labs (With Explanation of What You’re Seeing)

### Lab A: Read HTTP responses with `curl` (and interpret them)
`curl -i` prints headers + body.

1) 200 OK:

```bash
curl -i https://httpbin.org/status/200
```

What to notice:
- status line starts with `HTTP/... 200`
- headers follow
- body may be empty depending on endpoint

2) Redirect (3xx):

```bash
curl -i https://httpbin.org/redirect/1
```

What to notice:
- status is 302
- `Location` header tells the next URL
- browsers automatically follow; curl does not unless you use `-L`

3) Follow redirects:

```bash
curl -i -L https://httpbin.org/redirect/2
```

What to notice:
- you’ll see multiple responses
- final response typically is 200

### Lab B: Tiny Python server (to feel what Django abstracts)
This is not “how you should build web apps”. It’s to internalize what “a web
framework” provides.

Create `mini_server.py`:

```python
from http.server import BaseHTTPRequestHandler, HTTPServer
import json


class Handler(BaseHTTPRequestHandler):
    def _send_json(self, status_code, payload):
        body = json.dumps(payload).encode("utf-8")

        # Status line:
        self.send_response(status_code)

        # Headers:
        self.send_header("Content-Type", "application/json; charset=utf-8")
        self.send_header("Content-Length", str(len(body)))

        # End headers section:
        self.end_headers()

        # Body:
        self.wfile.write(body)

    def do_GET(self):
        # self.path includes the path + query string.
        # Example: "/hello?name=world"
        if self.path.startswith("/hello"):
            self._send_json(
                200,
                {
                    "message": "Hello from mini server",
                    "path_received": self.path,
                    "method": "GET",
                    "note": (
                        "This server is not parsing query parameters; "
                        "frameworks like Django do that for you."
                    ),
                },
            )
            return

        self._send_json(404, {"error": "Not found", "path_received": self.path})

    def do_POST(self):
        # For POST, the body is separate; length is described by a header:
        length = int(self.headers.get("Content-Length", "0"))
        raw = self.rfile.read(length) if length > 0 else b""

        self._send_json(
            200,
            {
                "message": "Received POST",
                "path_received": self.path,
                "content_type_received": self.headers.get("Content-Type"),
                "raw_body_received": raw.decode("utf-8", errors="replace"),
                "note": (
                    "In real frameworks, body parsing depends on Content-Type "
                    "(JSON vs form vs multipart)."
                ),
            },
        )


def main():
    server = HTTPServer(("127.0.0.1", 8001), Handler)
    print("Listening on http://127.0.0.1:8001")
    server.serve_forever()


if __name__ == "__main__":
    main()
```

Run:

```bash
python mini_server.py
```

Now test GET:

```bash
curl -i "http://127.0.0.1:8001/hello?name=world&page=2"
```

What you should understand:
- The server sees `self.path` as a string that includes `?name=world&page=2`
- It’s your responsibility (in raw servers) to parse it
- Django parses query parameters into a structured object for you

Test POST JSON:

```bash
curl -i -X POST http://127.0.0.1:8001/submit \
  -H "Content-Type: application/json" \
  --data '{"title":"Test","page":2}'
```

What you should understand:
- The body is read by length
- The meaning of the body depends on `Content-Type`
- Django automates reading/parsing and gives you `request.POST` and/or
  `request.body` / `request.data` (later with DRF)

---

## 1.13 Exercises (Answer in writing)

1) For this URL, label each part (scheme, host, path, query, fragment):

```text
https://shop.example.com/products/42/?page=2&sort=price#reviews
```

2) What does `?` mean in a URL? What does `&` mean?

3) Explain why pagination usually belongs in the query string rather than the path.

4) Describe two limitations of page-number pagination:
   - one correctness/UX issue
   - one performance/scaling issue

5) Explain the difference between:
- `Content-Type`
- `Accept`

6) Why do cookies increase CSRF risk?

---

## 1.14 Chapter Summary (What to retain permanently)

- HTTP is messages: method/path/headers/body → status/headers/body.
- `?` starts the query string; `&` separates parameters; fragments `#...` never reach
  the server.
- Pagination exists for performance + UX and is commonly query-based for composable
  URLs (`?page=2&sort=...`), but offset pagination has real limitations at scale.
- Cookies enable sessions, which enable login state—and also explain CSRF.
- Django’s design choices (CSRF protection, escaping, ORM) are direct responses to
  common web threats.

---

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <span style='color:gray; font-size:1.05em;'>Previous</span>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='2. python_for_django.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
